In [1]:
import io
import re
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

import six

from glob import glob
from google.cloud import vision

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [7]:
# Set Google API authentication and set folder where images are stored
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Banking-326c0d0e12c1.json'

client = vision.ImageAnnotatorClient()

In [8]:
all_images = glob(os.path.join('data', '*'))
df_images = pd.DataFrame({'path': all_images})
df_images.head()

,path
0,data\LIC1-1.jpg
1,data\LIC1-2.jpg
2,data\LIC1-3.jpg
3,data\LIC1-4.jpg
4,data\LIC1-5.jpg


In [9]:
df_images['file_name'] = df_images['path'].map(lambda in_path: in_path.split(os.sep)[-1])
df_images['file_type'] = df_images['path'].map(lambda in_path: os.path.splitext(in_path)[1][1:])  # [2]
df_images.head()

,path,file_name,file_type
0,data\LIC1-1.jpg,LIC1-1.jpg,jpg
1,data\LIC1-2.jpg,LIC1-2.jpg,jpg
2,data\LIC1-3.jpg,LIC1-3.jpg,jpg
3,data\LIC1-4.jpg,LIC1-4.jpg,jpg
4,data\LIC1-5.jpg,LIC1-5.jpg,jpg


In [10]:
def google_vision_it(row):
    with open(row.path, 'rb') as image_file:
        content = image_file.read()
    response = client.document_text_detection({'content': content})  # [1]
    texts = response.text_annotations
    return texts[np.argmax([len(t.description) for t in texts])].description.split('\n')

In [11]:
 df_images['rendered_text'] = df_images.apply(google_vision_it, axis=1)

In [9]:
df_result = df_images
df_result['corpus'] = df_images['rendered_text'].map(lambda l: ' '.join(l))
df_result

,path,file_name,file_type,rendered_text,entities,corpus
0,data\LIC1-1.jpg,LIC1-1.jpg,jpg,"[kotak life, Koi hai... hamesha, 072), .., o, ...","{'name': ['AS971331 Policy No', 'Mumbai', 'o P...",kotak life Koi hai... hamesha 072) .. o POLICY...
1,data\LIC1-2.jpg,LIC1-2.jpg,jpg,"[109, MAX, INSURANCE, INSURANCE PREMIUM RECEIP...","{'name': ['INSURANCE', 'Insured', 'Life', 'REC...",109 MAX INSURANCE INSURANCE PREMIUM RECEIPT Po...
2,data\LIC1-3.jpg,LIC1-3.jpg,jpg,"[HDFC, *, .-, Life, 1, Saiutha ke jiyo!, RENEW...","{'name': ['Satish Pagare', 'Life', 'HDFC', 'RE...",HDFC * .- Life 1 Saiutha ke jiyo! RENEWAL PREM...
3,data\LIC1-4.jpg,LIC1-4.jpg,jpg,"[LOR, lo, en, LIC, weinig om te, ISAARET bocht...","{'name': ['Collecting Branch', 'E-mail', 'Phon...",LOR lo en LIC weinig om te ISAARET bochtet Col...
4,data\LIC1-5.jpg,LIC1-5.jpg,jpg,"[22, LIFE INSURANCE, Aditya Birla Sun Life Ins...","{'name': ['Aditya Birla Sun Life Insurance', '...",22 LIFE INSURANCE Aditya Birla Sun Life Insura...
5,data\LIC1-6.jpg,LIC1-6.jpg,jpg,"[Emilia, rdi, GLIC ucal services., Life Insura...","{'name': ['Oly 926795440 Patil NN', 'ucal serv...",Emilia rdi GLIC ucal services. Life Insurance ...


# Helper Functions

In [10]:
 
def find_provider(corpus):
    insurance_providers = ['hdfc', 'lic','newindia', 'bajaj', 'birla', 'aegon','glic', 'sbi','pnb', 'exide','icici','max','kotak','bupa','apollo','bharti','magma','liberty','generali','oriental','star','royal','aig','reliance','universal','united','shriram','cigna','raheja','cholamandalam','ecgc','iffco','aviva','pnb','idbi','indiafirst','peerless','sahara','new india','religare','hsbc','dhfl','edelweiss','aia','dai-ichi','sundaram','l&t','new india','GLIC','star','http://newindia.co.in.']
    corpus_split = corpus.lower().split(' ')
    for provider in insurance_providers:
        if provider in corpus_split:
            return provider
        
    return None   
 
def find_provider_reg(corpus):
    regex=r"(Apollo Munich|Apollo Munich|SBI Life|SBI|sbi|EXIDE Life|ICICI PRUDENTIAL|ICICI Lombard|ICICI LOMBARD|Max Life|max life|HDFC Life|LIC of India|Life Insurance Corporation|LIFE INSURANCE CORPORATION|LIC|kotak life|Kotak Life|Kotak Mahindra Life|STAR COMPREHENSIVE INSURANCE|Star Health|NEW INDIA ASSURANCE|NEW INDIA|HDFC ERGO|HDFC ERGO General Insurance|Aegon Life|Bajaj Allianz|Birla Sun Life|Birla Sun Life Insurance)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()
    return None



In [11]:
def find_policy(corpus):
    #regex = r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master olicy)+\s(No|Num|No.|no.|num|Number|number) \d+"
    regex=r" (Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Details|Proposal/Policy|Policy|)+\s(No|Num|No.|no.|num|Number|number)(\s|\s+|.|)(:|,|;|-|=|\s|) (\d\w\d+|\d+\w+\d+)"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

def find_policy_slash(corpus):
    regex=r" (Policy.No....(\w+|\d+)(.|)(\w+|\d+)(.|)(\w+|\d+)(.|)(\w+|\d+)(.|)(\w+|\d+))"
    matches = re.finditer(regex, corpus, re.MULTILINE)
    for match in matches:
        return match.group()

In [12]:
def find_premium(rendered_text):
    regex = r"(Premium.A.|Premium Amount|Premium amount|Premium|Sum Assured|Total Premium Amount|Amount paid|Total Premium Paid|Gross Premium .Rs.|Gross)..(?: \d\d+)+ (?=\()|\d+,\d+.\d\d"
    for text in rendered_text:
        matches = re.match(regex, text)
        if matches:
            return text
            break
            
    return None

In [13]:
def find_name(rendered_text):
    regex = r"(Mr./Mrs.|Mr|MR|Mr.|Ms|Ms.|Miss|Mrs|Mrs.|Prof|Mr./Mrs.|Name:|Dear MR.|Prof.|Dr|Dr.|Smt.|Ms.|Shri|Sri|Sri.|MR |Policy Holder Name|Premium Payor Name|Policyholder's Name|Life Insured..|Owner Name...|Dear Mr.|following life insurance policy held by|The following premium has been received for life insurance policies from the userid of)(.|..|...|)((\w+ \w+ \w+ \w+)|(\w\w+)+|(\w \w+ \w+))"
    for text in rendered_text:
        matches = re.match(regex, text)
        if matches:
            return text
            break 
            
    return None

In [14]:
def find_date(rendered_text):
    regex = r"((Date|Date |DATE|date|Date of issue|DATE OF ISSUE)(\s|):+.(\d+|\w+|\w)(-|/|\|.|,|\s)(\d+|\w+)(-|/|\|.|,|\s)(\d+|\w+))"
    d = None
    for text in rendered_text:
        matches = re.match(regex, text)
        if matches:
            return text
        
    return None

In [15]:

def valid_premium_word(premium_word):
    try:
        mat=re.match("^((Premium.A.|Premium Amount|Premium amount|Installment premium|Installment Premium|Amt. Collected|Total Premium|Gross Premium .Rs.|Gross))$",premium_word)
        return mat.group()
    except ValueError:
        return False
    except AttributeError:
        return False

def valid_premium_no(premium_no):
    try:
        mat=re.match("^(?: \d\d+)+ (?=\()|\d+|\d+,\d+.\d\d$",premium_no)
        return mat.group()
    except ValueError:
        return False
    except AttributeError:
        return False

def find_premium_ex(corpus):
    words = corpus.split(' ')
    for i in range(0, len(words)):
        if words[i] == valid_premium_word(words[i]):
            for j in range(i, len(words)):
                if words[j] == valid_premium_no(words[j]):
                    return words[j]      
    return None
                

def find_premium_back(rendered_text):
    for i in range(0,len(rendered_text)):
        if rendered_text[i] == valid_premium_word(rendered_text[i]):
            k=i
            for j in range(k, 0, -1):
                 if rendered_text[j] == valid_premium_no(rendered_text[j]):
                        return rendered_text[j]
                        break
    return None

def prem_amt(corpus):
    regex = r"(Premium.A.|Premium Amount|Premium amount|Sum Assured|Total Premium Amount|Amount Paid|Total Premium Paid|Total|Total Amount Rs. |amount of|Amount paid|premium amount of Rs.|Total Premium :Rs|Amt. Collected) ..((\d+.\d\d)|(: \d+.\d\d)|(\d+,\d+.\d\d))"
    
    test_str = corpus
    matches = re.finditer(regex, test_str, re.MULTILINE)

    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1
        if match:
            return match.group()
    return None

def prem_amt1(corpus):
    regex = r"(Premium.A.|Premium Amount|Premium amount|Sum Assured|Total Premium Amount|Amount Paid|Total Premium Paid|Total|Total Amount Rs. |amount of|Amount paid|premium amount of Rs.|Amt. Collected|Total Premium :Rs|paid...) ((\d+,\d+.\d\d)|(\d+.\d\d)|(\d+))"
    
    test_str = corpus
    matches = re.finditer(regex, test_str, re.MULTILINE)

    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1
        if match:
            return match.group()
    return None

def prem_recpt(corpus):
    regex = r"(receipt[^\d]+\d+)"
    
    test_str = corpus
    matches = re.finditer(regex, test_str, re.MULTILINE)

    for matchNum, match in enumerate(matches):
        matchNum = matchNum + 1
        if match:
            return match.group()
    return None

In [16]:
def valid_policy_word(policy_word):
    try:
        mat=re.match("^((Policy|policy|P.|p.|P|p|Pol|pol|Pol.|pol.|Master|1 plicy|licy|olicy|icy)+\s(No|Num|No.|no.|num|Number|number|No.Plan|Policy :|Number:))$",policy_word)
        return mat.group()
    except ValueError:
        return False
    except AttributeError:
        return False

def valid_policy_no(policy_no):
    try:
        mat=re.match("(\d\w\d+|\d+\w+\d+|^\d+$)",policy_no)
        #mat=re.match("^\d+$",policy_no)
        return mat.group()
    except ValueError:
        return False
    except AttributeError:
        return False

def find_policy_ex(rendered_text):            
    for i in range(0, len(rendered_text)):
        if rendered_text[i] == valid_policy_word(rendered_text[i]):
            for j in range(i, len(rendered_text)):
                if rendered_text[j] == valid_policy_no(rendered_text[j]):
                    return rendered_text[j]
    return None


def find_policy_back(rendered_text):
    for i in range(0,len(rendered_text)):
        if rendered_text[i] == valid_policy_word(rendered_text[i]):
            k=i
            for j in range(k, 0, -1):
                 if rendered_text[j] == valid_policy_no(rendered_text[j]):
                        return rendered_text[j]
                        break
    return None

def find_policy_sep_number(rendered_text):
    for i in range(0,len(rendered_text)):
        if rendered_text[i]=='Policy'and rendered_text[i+1]=='Number':
            for j in range(i, len(rendered_text)):
                    if rendered_text[j] == valid_policy_no(rendered_text[j]):
                        return rendered_text[j]
        elif rendered_text[i]=='policy'and rendered_text[i+1]=='number':
            for j in range(i, len(rendered_text)):
                    if rendered_text[j] == valid_policy_no(rendered_text[j]):
                        return rendered_text[j]
        elif rendered_text[i]=='Policy'and rendered_text[i+1]=='number':
            for j in range(i, len(rendered_text)):
                    if rendered_text[j] == valid_policy_no(rendered_text[j]):
                        return rendered_text[j]
    return None

def find_single_policy(corpus):
    for i in corpus:
        x=re.search("((Policy....)(\d+))",corpus)
        if x:
            return x.group()
            break
    return None

In [17]:
def valid_dateword(date_word):
    try:
        mat=re.match("^((Date|Date:|DATE|date|Date of issue|DATE OF ISSUE|Date and Time :|Receipt Date))$",date_word)
        return mat.group()
    except ValueError:
        return False
    except AttributeError:
        return False

def valid_date(date_string):
    try:
        mat=re.match("^(\d+|\w+|\w)(-|/|\|.|,|\s)(\d+|\w+)(-|/|\|.|,|\s)(\d+|\w+)$",date_string)
        return mat.group()
    except ValueError:
        return False
    except AttributeError:
        return False

def find_date_ex(rendered_text):    
    for i in range(0,len(rendered_text)):
        if rendered_text[i]==valid_dateword(rendered_text[i]):
            for j in range(i, len(rendered_text)):
                if rendered_text[j] == valid_date(rendered_text[j]):
                    return rendered_text[j]
    return None 

def find_date_single(rendered_text):
    for i in rendered_text:
        x=re.search("^(0?[1-9]|1[0-9]|2[0-9]|3[0-1])([. \/-])(0?[1-9]|1[0-2]|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)|Apr(?:il)|May|Jun(?:ne)|Jul(?:y)|Aug(?:)|Sep(?:)|Oct(?:)|Nov(?:)|Dec(?:ember)?)|(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)|apr(?:il)|may|jun(?:ne)|jul(?:y)|aug(?:)|sep(?:)|Oct(?:)|nov(?:)|dec(?:ember)?))[(. \-/](19[0-9][0-9]|20[0-9][0-9])$",i)
        if x:
            return x.group()
        elif x is None: 
                x = re.search('(\w+)(\s)(\d+)(,)(\s|)(\d+)', i)
                if x:
                    return x.group()
    return None 

def find_if_date_and_time(corpus):
    for i in corpus:
        x=re.search("((Date...Time.|Receipt Date)(\s|)(|:|)+.(\d+|\w+|\w)(-|/|\|.|,|\s)(\d+|\w+)(-|/|\|.|,|\s)(\d+|\w+))",corpus)
        if x:
            return x.group()
            break
    return None

def find_if_date_time(corpus):
    for i in corpus:
        x=re.search("((Date and Time)(\s|)(:)(\s|)(\d+|\w+|\w)(-|/|\|.|,|\s)(\d+|\w+)(-|/|\|.|,|\s)(\d+|\w+))",corpus)
        if x:
            return x.group()
            break
    return None

# Main function

In [18]:
def information_extract(row):
    rendered = row.rendered_text
    corpus = row.corpus
    
    provider = find_provider_reg(corpus)
    if provider is None:
        find_provider(corpus)
    
    policy_number = find_policy(corpus)
    if policy_number is None:
        policy_number = find_policy_ex(rendered)
        policy_number=policy_number
        if policy_number is None:
            policy_number=find_policy_back(rendered)
            policy_number=policy_number
            if policy_number is None:
                policy_number = find_policy_sep_number(rendered)
                policy_number=policy_number
                if policy_number is None:
                    policy_number = find_single_policy(corpus)
                    policy_number=policy_number
                    if policy_number is None:
                        policy_number = find_policy_slash(corpus)
        
            
        
    premium_amt = find_premium(rendered)
    if premium_amt is None:
        premium_amt = find_premium_ex(corpus)
        premium_amt=premium_amt
        if premium_amt is None:
            premium_amt = prem_amt(corpus)
            premium_amt=premium_amt
            if premium_amt is None:
                premium_amt = prem_amt1(corpus)
                premium_amt=premium_amt
                if premium_amt is None:
                    premium_amt=find_premium_back(rendered)
                    premium_amt=premium_amt
                    if premium_amt is None:
                        premium_amt = prem_recpt(corpus)
            
        
    insured_name = find_name(rendered)
    # insured_name = natural_language(insured_name)
    
    
    premium_date = find_if_date_and_time(corpus)   
    premium_date=premium_date
    if premium_date is None:
        premium_date = find_date(rendered)
        premium_date=premium_date
        if premium_date is None:
            premium_date = find_date_ex(rendered)
            premium_date=premium_date
            if premium_date is None:
                premium_date=find_date_single(rendered)
                premium_date=premium_date
                if premium_date is None:
                    premium_date = find_if_date_time(corpus)
    
            
                

             
    return pd.Series({'provider':provider, 'policy_number':policy_number, 'premium_amt' : premium_amt, 'insured_name': insured_name, 'premium_date': premium_date})

In [19]:
df_final_result = pd.concat([df_result, df_result.apply(information_extract, axis=1)], axis=1)

In [20]:
df_final_result[['file_name', 'provider', 'policy_number' , 'premium_amt', 'insured_name', 'premium_date']]

,file_name,provider,policy_number,premium_amt,insured_name,premium_date
0,LIC1-1.jpg,kotak life,Policy No : 00862916,receipt of Rs. **********10000,Life Insured: Arvind Yeshwant Sawant,Receipt Date : 15/01/2018
1,LIC1-2.jpg,Max Life,Policy Number: 406769299,"16,841.23",Mr. Ansley Frank Sheath,29-Jun-2017
2,LIC1-3.jpg,HDFC Life,Proposal/Policy No.: 16723771,"21,800.00","Dear Mr. Satish Pagare,",19/01/2017
3,LIC1-4.jpg,LIC,991831396,"13,150.00",Smt./Ms./Shri : Yash Shashikant Lakhani,Date ( Time: 04/12/2016
4,LIC1-5.jpg,Birla Sun Life,Policy Number: 005931330,"premium amount of Rs. 10,991.38",Owner Name : YASH SASHIKANT LAKHANI,Date: 04/01/2018 14:51
5,LIC1-6.jpg,LIC,883527669,"3,062.00",The following premium has been received for li...,DATE OF ISSUE :22/01/2018


In [21]:
words = ['http','Mrs.','Policy','Premium Payor','of issue','Number','Details','No.','No','Mr.','MR ','MR.','.co.in.','Owner Name','Receipt','Rs.','Dear',':','/Premium Paying','holder','Name','Life Insured','Premium Payor Name','Holder Name','Proposal/','Date','DATE','OF','ISSUE','Date','and','Time','DATE OF ISSUE','Mobile','Smt./Ms./Shri','Holder','The following premium has been received for life insurance policies from the userid of','following life insurance policy held by'] 
for i in words:
    df_final_result = df_final_result.replace(to_replace=i,value="",regex=True)
    

df_final_result['provider']=df_final_result['provider'].str.replace(r"[\(\)\{\}<>/]","")
df_final_result['policy_number']=df_final_result['policy_number'].str.replace(r"[\(\)\{\}<>|]","")
df_final_result['insured_name']=df_final_result['insured_name'].str.replace(r"[\(\)\{\}<>,']","")
df_final_result['premium_date']=df_final_result['premium_date'].str.replace(r"[\(\)\{\}<>]","")


k=0
for i in df_final_result['premium_amt']:
    if i != None:
        df_final_result.loc[k,'premium_amt']=re.sub('[^0-9,.]', "", df_final_result.loc[k,'premium_amt'])
    k=k+1


In [22]:
def trimAllColumns(df_final_result):
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df_final_result.applymap(trimStrings)

df_final_result = trimAllColumns(df_final_result)
df_final_result[['file_name', 'provider', 'policy_number' , 'premium_amt', 'insured_name', 'premium_date']]

,file_name,provider,policy_number,premium_amt,insured_name,premium_date
0,LIC1-1.jpg,kotak life,00862916,10000,Arvind Yeshwant Sawant,15/01/2018
1,LIC1-2.jpg,Max Life,406769299,"16,841.23",Ansley Frank Sheath,29-Jun-2017
2,LIC1-3.jpg,HDFC Life,16723771,"21,800.00",Satish Pagare,19/01/2017
3,LIC1-4.jpg,LIC,991831396,"13,150.00",Yash Shashikant Lakhani,04/12/2016
4,LIC1-5.jpg,Birla Sun Life,005931330,"10,991.38",YASH SASHIKANT LAKHANI,04/01/2018 1451
5,LIC1-6.jpg,LIC,883527669,"3,062.00",Nikhil N Patil,22/01/2018


In [23]:
for i in range(0,len(df_final_result)):
    try:
        df_final_result.loc[i,'premium_date'] = pd.to_datetime(df_final_result.loc[i,'premium_date'],dayfirst=True)
    except ValueError:
        df_final_result.loc[i,'premium_date'] = None

df_final_result['premium_date'] = pd.to_datetime(df_final_result['premium_date'],dayfirst=True).dt.date
df_final_result[['file_name', 'provider', 'policy_number' , 'premium_amt', 'insured_name', 'premium_date']]

,file_name,provider,policy_number,premium_amt,insured_name,premium_date
0,LIC1-1.jpg,kotak life,00862916,10000,Arvind Yeshwant Sawant,2018-01-15
1,LIC1-2.jpg,Max Life,406769299,"16,841.23",Ansley Frank Sheath,2017-06-29
2,LIC1-3.jpg,HDFC Life,16723771,"21,800.00",Satish Pagare,2017-01-19
3,LIC1-4.jpg,LIC,991831396,"13,150.00",Yash Shashikant Lakhani,2016-12-04
4,LIC1-5.jpg,Birla Sun Life,005931330,"10,991.38",YASH SASHIKANT LAKHANI,2018-01-04
5,LIC1-6.jpg,LIC,883527669,"3,062.00",Nikhil N Patil,2018-01-22


In [24]:
writer = ExcelWriter('output.xlsx')
df_final_result[['file_name', 'provider', 'policy_number' , 'premium_amt', 'insured_name', 'premium_date']].to_excel(writer,'Sheet1',index=False)
writer.save()